In [10]:
import os
import sys
import time
import numpy as np
import pygame
from gymnasium import Env
from gymnasium.spaces import Discrete, Box

sys.path.insert(1, os.path.join(sys.path[0], '..'))
from Agents.agent import Agent
from Agents.RayCast import get_fov_rays
from Constants.constants import WHITE, RED, BLUE, SCREEN_WIDTH, SCREEN_HEIGHT, WALLS, WALLS2
from Walls.collision_detection import detect_collision
from Walls.wall_class import Walls




In [23]:

class GameEnv(Env):
    def __init__(self, render_mode='human'):
        super(GameEnv, self).__init__()

        # defining the screen dimension for render purpose
        self.screen_width = SCREEN_WIDTH
        self.screen_height = SCREEN_HEIGHT
        self.render_mode = render_mode

        total_values = 219
        self.observation_space = Box(
            low=np.zeros(total_values, dtype=np.float32),
            high=self.screen_width * np.ones(total_values, dtype=np.float32),
            dtype=np.float32
        )

        # Define the radius for the semi-circular reward area
        self.semi_circle_radius = 250  # You can adjust this radius

        #! Define the center of the screen for the green dot
        self.center_x = self.screen_width // 2
        self.center_y = self.screen_height // 2

        self.action_space = Discrete(5)

        self.total_steps = 0

        self.number_of_predator = 1

        self.predator_agent = None

        self.predator_total_reward = 0

        self.obs = None

        # start the tick timer
        self.start_time = 0
        self.total_running_time = 10

        pygame.init()
        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        pygame.display.set_caption('Multi Agent Environment(simple)')

        pygame.font.init()
        self.font = pygame.font.Font(None, 18)

        self.wall = Walls(pygame)
        self.walls = None

    def agent_init(self):
        predator_agents = Agent('predator', 0)
        self.predator_agent = predator_agents

    def flatten_list(self, nested_list):
        flattened_list = []
        for item in nested_list:
            if isinstance(item, list):
                flattened_list.extend(self.flatten_list(item))
            else:
                flattened_list.append(item)
        return flattened_list
    
    def _get_obs(self):
        observation = []
        agent_pos = [self.predator_agent.current_position[0], self.predator_agent.current_position[1]]
        observation.append(agent_pos)

        angle = self.predator_agent.angle
        observation.append(angle)

        value_list = get_fov_rays(agent_pos)
        observation.append(value_list)
        
        observation = self.flatten_list(observation)
        return observation

    def _max_right(self):
        max_right = 0
        for wall in self.walls:
            if wall.right > max_right:
                max_right = wall.right
        return max_right

    def reset(self, seed=0):
        self.start_time = time.time()
        self.agent_init()
        self.wall.clear_walls()
        self.walls = self.wall.make_wall(WALLS2)
        self.total_steps = 0
        self.predator_total_reward = 0

        predator = self.predator_agent
        predator.agent_reset(width=self.screen_width, height=self.screen_height, walls=self.walls)

        self.predator_agent = predator

        observation = self._get_obs()
        self.obs = observation

        return observation, seed

    def step(self, action):
        done = False
        reward = 0
        truncated = False
        info = {}
        current_time = time.time()

        elapsed_time = current_time - self.start_time
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
                pygame.quit()
        self.predator_agent.step_update(action, range_x=self.screen_width, range_y=self.screen_height)
        self.predator_agent = detect_collision(self.predator_agent, self.walls)

        # Calculate the center of the exponential reward area
        center_x, center_y = 325, 300
        # Calculate the distance from the agent to the center
        distance_to_center = ((self.predator_agent.current_position[0] - center_x) ** 2 + (self.predator_agent.current_position[1] - center_y) ** 2) ** 0.5
        # Set the radius for the exponential reward function to start working
        radius_for_exponential_reward = 100  # Adjust this radius as needed
        # Check if the agent is within the exponential reward area
        if distance_to_center <= radius_for_exponential_reward:
            reward = 10 * np.exp(-1 * distance_to_center / 100) + 0.05

        if self.predator_agent.current_position[0] > self._max_right():
            reward += 100
            done = True

        if elapsed_time >= self.total_running_time:
            reward -= 50
            done = True

        reward += 0.01
        self.render()

        observation = self._get_obs()
        self.predator_total_reward = reward
        self.obs = observation

        return observation, reward, done, truncated, info

    def render(self):
        if self.render_mode == 'human':
            screen = self.screen
            screen.fill(WHITE)
            
            predator = self.predator_agent
            pygame.draw.circle(screen, RED, predator.center, predator.radius)
            pygame.draw.line(screen, RED, predator.center, predator.draw_direction_end, 5)

            for key, wall in WALLS2.items():
                pygame.draw.rect(screen, BLUE, (wall['x'], wall['y'], wall['width'], wall['height']))

            max_right = self._max_right()
            pygame.draw.circle(screen, (0, 255, 0), (325, 300), 5)

            max_right_color = (255, 0, 0)
            pygame.draw.line(self.screen, max_right_color, (max_right, 0), (max_right, self.screen_height), 2)

            pygame.display.update()

    def close(self):
        pygame.quit()


In [24]:
env = GameEnv()

In [25]:
res = env.reset()

In [26]:
from stable_baselines3 import PPO

In [27]:
log_path = os.path.join('Training', 'Logs', 'Level_01_PPO')
baseline_path = os.path.join('Training', 'Models', 'Level_01_PPO')

In [28]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)
env.reset()
model.learn(total_timesteps=50000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to Training\Logs\Level_01_PPO\PPO_12


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 667      |
|    ep_rew_mean     | -43.3    |
| time/              |          |
|    fps             | 66       |
|    iterations      | 1        |
|    time_elapsed    | 30       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 629         |
|    ep_rew_mean          | -43.7       |
| time/                   |             |
|    fps                  | 62          |
|    iterations           | 2           |
|    time_elapsed         | 65          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010956144 |
|    clip_fraction        | 0.0685      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.6        |
|    explained_variance   | 0.00521     |
|    learning_rate        | 0.

In [29]:
env.close()

In [30]:
model.save(baseline_path)


In [31]:
%tensorboard

UsageError: Line magic function `%tensorboard` not found.


In [1]:
import os
import sys
import time
import numpy as np
import pygame
from gymnasium import Env
from gymnasium.spaces import Discrete, Box

sys.path.insert(1, os.path.join(sys.path[0], '..'))
from Agents.agent import Agent
from Agents.RayCast import get_fov_rays
from Constants.constants import WHITE, RED, BLUE, SCREEN_WIDTH, SCREEN_HEIGHT, WALLS, WALLS2
from Walls.collision_detection import detect_collision
from Walls.wall_class import Walls

class GameEnv(Env):
    def __init__(self, render_mode='human'):
        super(GameEnv, self).__init__()

        # defining the screen dimension for render purpose
        self.screen_width = SCREEN_WIDTH
        self.screen_height = SCREEN_HEIGHT
        self.render_mode = render_mode

        total_values = 219
        self.observation_space = Box(
            low=np.zeros(total_values, dtype=np.float32),
            high=self.screen_width * np.ones(total_values, dtype=np.float32),
            dtype=np.float32
        )

        # Define the radius for the semi-circular reward area
        self.semi_circle_radius = 250  # You can adjust this radius

        #! Define the center of the screen for the green dot
        self.center_x = self.screen_width // 2
        self.center_y = self.screen_height // 2

        self.action_space = Discrete(5)

        self.total_steps = 0

        self.number_of_predator = 1

        self.predator_agent = None

        self.predator_total_reward = 0

        self.obs = None

        # start the tick timer
        self.start_time = 0
        self.total_running_time = 10

        pygame.init()
        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        pygame.display.set_caption('Multi Agent Environment(simple)')

        pygame.font.init()
        self.font = pygame.font.Font(None, 18)

        self.wall = Walls(pygame)
        self.walls = None

    def agent_init(self):
        predator_agents = Agent('predator', 0)
        self.predator_agent = predator_agents

    def flatten_list(self, nested_list):
        flattened_list = []
        for item in nested_list:
            if isinstance(item, list):
                flattened_list.extend(self.flatten_list(item))
            else:
                flattened_list.append(item)
        return flattened_list
    
    def _get_obs(self):
        observation = []
        agent_pos = [self.predator_agent.current_position[0], self.predator_agent.current_position[1]]
        observation.append(agent_pos)

        angle = self.predator_agent.angle
        observation.append(angle)

        value_list = get_fov_rays(agent_pos)
        observation.append(value_list)
        
        observation = self.flatten_list(observation)
        return observation

    def _max_right(self):
        max_right = 0
        for wall in self.walls:
            if wall.right > max_right:
                max_right = wall.right
        return max_right

    def reset(self, seed=0):
        self.start_time = time.time()
        self.agent_init()
        self.wall.clear_walls()
        self.walls = self.wall.make_wall(WALLS2)
        self.total_steps = 0
        self.predator_total_reward = 0

        predator = self.predator_agent
        predator.agent_reset(width=self.screen_width, height=self.screen_height, walls=self.walls)

        self.predator_agent = predator

        observation = self._get_obs()
        self.obs = observation

        return observation, seed

    def step(self, action):
        done = False
        reward = 0
        truncated = False
        info = {}
        current_time = time.time()

        elapsed_time = current_time - self.start_time
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
                pygame.quit()
        self.predator_agent.step_update(action, range_x=self.screen_width, range_y=self.screen_height)
        self.predator_agent = detect_collision(self.predator_agent, self.walls)

        self.total_steps += 1
        # Calculate the distance to the center (325, 300)
        center_x = 325
        center_y = 300
        distance_to_center = ((self.predator_agent.current_position[0] - center_x) ** 2 + (self.predator_agent.current_position[1] - center_y) ** 2) ** 0.5
        # Use the given exponential reward function
        reward = 100 * np.exp(-2 * distance_to_center / 100) - 1.5
         
        if self.predator_agent.current_position[0] > self._max_right():
            reward += 100
            done = True

        if elapsed_time >= self.total_running_time:
            reward -= 50
            done = True
      
        reward += 0.01
        self.render()

        observation = self._get_obs()
        self.predator_total_reward = reward
        self.obs = observation

        return observation, reward, done, truncated, info

    def render(self):
        if self.render_mode == 'human':
            screen = self.screen
            screen.fill(WHITE)
            
            predator = self.predator_agent
            pygame.draw.circle(screen, RED, predator.center, predator.radius)
            pygame.draw.line(screen, RED, predator.center, predator.draw_direction_end, 5)

            for key, wall in WALLS2.items():
                pygame.draw.rect(screen, BLUE, (wall['x'], wall['y'], wall['width'], wall['height']))


            # Calculate the center of the exponential reward area
            center_x, center_y = 325, 300
            # Calculate the distance from the agent to the center
            distance_to_center = ((self.predator_agent.current_position[0] - center_x) ** 2 + (self.predator_agent.current_position[1] - center_y) ** 2) ** 0.5
            # Check if the agent is within the exponential reward area
            if distance_to_center <= 250:  # Adjust this radius as needed
                reward = 100 * np.exp(-2 * distance_to_center / 100) - 1.5


            max_right = self._max_right()
            pygame.draw.circle(screen, (0, 255, 0), (325, 300), 5)

            max_right_color = (255, 0, 0)
            pygame.draw.line(self.screen, max_right_color, (max_right, 0), (max_right, self.screen_height), 2)

            pygame.display.update()

    def close(self):
        pygame.quit()

def main():
    # Initialize the environment
    env = GameEnv(render_mode='human')

    # Reset the environment to get the initial observation
    observation, seed = env.reset()

    done = False
    while not done:
        # Replace this with your agent's logic to choose actions
        # In this example, we're taking a random action.
        action = env.action_space.sample()

        # Step through the environment with the chosen action
        observation, reward, done, truncated, info = env.step(action)

    # Close the environment when done
    env.close()

if __name__ == "__main__":
    main()


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [43]:
import pygame
import sys
import math

# Initialize Pygame
pygame.init()

# Set up display
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Predator Agent Environment")

# Define colors
RED = (255, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# Predator agent position and size
predator_x, predator_y = WIDTH // 2, HEIGHT // 2
predator_radius = 10
move_step = 1

# Define objects with arbitrary shapes
objects = {
    "special_wall": [(100, 100), (120, 100), (120, 200), (100, 200)],
    "wall2": [(200, 300), (220, 300), (220, 400), (200, 400)],
    "wall3": [(300, 200), (320, 200), (320, 300), (300, 300)],
    "circular_wall": [(400, 400), (430, 400), (430, 430), (400, 430)]
}

def is_ray_blocked(ray_start, ray_angle):
    closest_intersection = None

    x1, y1 = ray_start
    x2, y2 = x1 + 1000 * math.cos(math.radians(ray_angle)), y1 + 1000 * math.sin(math.radians(ray_angle))

    for object_name, object_vertices in objects.items():
        for i in range(len(object_vertices)):
            x3, y3 = object_vertices[i]
            x4, y4 = object_vertices[(i + 1) % len(object_vertices)]

            denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

            if denominator == 0:
                continue

            t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denominator
            u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denominator

            epsilon = 1e-5  # Small epsilon value

            if -epsilon <= t <= 1 + epsilon and 0 <= u <= 1:
                intersection_x = x1 + t * (x2 - x1)
                intersection_y = y1 + t * (y2 - y1)

                distance = math.hypot(intersection_x - x1, intersection_y - y1)
                if closest_intersection is None or distance < closest_intersection:
                    closest_intersection = distance

    return closest_intersection

# Main game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        predator_x -= move_step
    if keys[pygame.K_RIGHT]:
        predator_x += move_step
    if keys[pygame.K_UP]:
        predator_y -= move_step
    if keys[pygame.K_DOWN]:
        predator_y += move_step

    screen.fill(WHITE)

    for angle in range(360):
        ray_start = (predator_x, predator_y)
        ray_angle = angle
        closest_intersection = is_ray_blocked(ray_start, ray_angle)

        if closest_intersection is not None:
            ray_end_x = ray_start[0] + closest_intersection * math.cos(math.radians(ray_angle))
            ray_end_y = ray_start[1] + closest_intersection * math.sin(math.radians(ray_angle))
            pygame.draw.line(screen, RED, ray_start, (ray_end_x, ray_end_y), 1)
        else:
            ray_end_x = ray_start[0] + 1000 * math.cos(math.radians(ray_angle))
            ray_end_y = ray_start[1] + 1000 * math.sin(math.radians(ray_angle))
            pygame.draw.line(screen, BLUE, ray_start, (ray_end_x, ray_end_y), 1)

    pygame.draw.circle(screen, RED, (predator_x, predator_y), predator_radius)

    for object_name, object_vertices in objects.items():
        pygame.draw.polygon(screen, GREEN, object_vertices)
        

    pygame.display.flip()

pygame.quit()
sys.exit()


SystemExit: 

In [46]:
import pygame
import sys
import math

# Initialize Pygame
pygame.init()

# Set up display
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Predator Agent Environment")

# Define colors
RED = (255, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)  # Define green color
BLUE = (0, 0, 255)   # Define blue color for vision rays

# Predator agent position and size
predator_x, predator_y = WIDTH // 2, HEIGHT // 2
predator_radius = 10  # Radius of the circular predator agent
move_step = 1  # Adjust the movement step

# Define walls in a dictionary
walls = {
    "special_wall": {"x": 100, "y": 100, "width": 20, "height": 100},
    "wall2": {"x": 200, "y": 300, "width": 20, "height": 100},
    "wall3": {"x": 300, "y": 200, "width": 20, "height": 100},
}

# Function to check ray-wall intersection
def is_ray_blocked(ray_start, ray_angle):
    closest_intersection = None
    blocking_wall = None

    x1, y1 = ray_start
    x2, y2 = x1 + 1000 * math.cos(math.radians(ray_angle)), y1 + 1000 * math.sin(math.radians(ray_angle))

    for wall_name, wall in walls.items():
        x3, y3 = wall["x"], wall["y"]
        x4, y4 = x3 + wall["width"], y3 + wall["height"]

        denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

        if denominator == 0:
            continue

        t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denominator
        u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denominator

        if 0 <= t <= 1 and 0 <= u <= 1:
            intersection_x = x1 + t * (x2 - x1)
            intersection_y = y1 + t * (y2 - y1)

            if not closest_intersection or t < closest_intersection:
                closest_intersection = t
                blocking_wall = wall_name

    return closest_intersection, blocking_wall

# Main game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Handle user input
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        predator_x -= move_step
    if keys[pygame.K_RIGHT]:
        predator_x += move_step
    if keys[pygame.K_UP]:
        predator_y -= move_step
    if keys[pygame.K_DOWN]:
        predator_y += move_step

    # Clear the screen
    screen.fill(WHITE)

    # Draw vision rays
    for angle in range(360):
        ray_start = (predator_x, predator_y)
        ray_angle = angle
        ray_end = (ray_start[0] + 1000 * math.cos(math.radians(ray_angle)),
                   ray_start[1] + 1000 * math.sin(math.radians(ray_angle)))

        # Check if the ray is blocked by any wall
        closest_intersection, blocking_wall = is_ray_blocked(ray_start, ray_angle)

        if closest_intersection is not None:
            ray_end_x = ray_start[0] + closest_intersection * (ray_end[0] - ray_start[0])
            ray_end_y = ray_start[1] + closest_intersection * (ray_end[1] - ray_start[1])

            if blocking_wall == "special_wall":
                color = RED
            else:
                color = GREEN

            pygame.draw.line(screen, color, ray_start, (ray_end_x, ray_end_y), 1)
        else:
            pygame.draw.line(screen, BLUE, ray_start, ray_end, 1)

    # Draw the predator agent as a circle
    pygame.draw.circle(screen, RED, (predator_x, predator_y), predator_radius)

    # Draw walls from the dictionary with green color
    for wall in walls.values():
        pygame.draw.rect(screen, GREEN, (wall["x"], wall["y"], wall["width"], wall["height"]))

    # Update the display
    pygame.display.flip()

# Quit Pygame
pygame.quit()
sys.exit()


SystemExit: 

In [49]:
import pygame
import sys
import math

class GameEnv:
    def __init__(self, width, height):
        pygame.init()
        self.WIDTH = width
        self.HEIGHT = height
        self.screen = pygame.display.set_mode((self.WIDTH, self.HEIGHT))
        pygame.display.set_caption("Predator Agent Environment")

        # Define colors
        self.RED = (255, 0, 0)
        self.WHITE = (255, 255, 255)
        self.GREEN = (0, 255, 0)
        self.BLUE = (0, 0, 255)

        # Predator agent position and size
        self.predator_x, self.predator_y = self.WIDTH // 2, self.HEIGHT // 2
        self.predator_radius = 10
        self.move_step = 1

        # Define objects with arbitrary shapes
        self.objects = {
            "special_wall": [(100, 100), (120, 100), (120, 200), (100, 200)],
            "wall2": [(200, 300), (220, 300), (220, 400), (200, 400)],
            "wall3": [(300, 200), (320, 200), (320, 300), (300, 300)],
            "circular_wall": [(400, 400), (430, 400), (430, 430), (400, 430)]
        }

    def is_ray_blocked(self, ray_start, ray_angle):
        closest_intersection = None

        x1, y1 = ray_start
        x2, y2 = x1 + 1000 * math.cos(math.radians(ray_angle)), y1 + 1000 * math.sin(math.radians(ray_angle))

        for object_name, object_vertices in self.objects.items():
            for i in range(len(object_vertices)):
                x3, y3 = object_vertices[i]
                x4, y4 = object_vertices[(i + 1) % len(object_vertices)]

                denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

                if denominator == 0:
                    continue

                t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denominator
                u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denominator

                epsilon = 1e-5  # Small epsilon value

                if -epsilon <= t <= 1 + epsilon and 0 <= u <= 1:
                    intersection_x = x1 + t * (x2 - x1)
                    intersection_y = y1 + t * (y2 - y1)

                    distance = math.hypot(intersection_x - x1, intersection_y - y1)
                    if closest_intersection is None or distance < closest_intersection:
                        closest_intersection = distance

        return closest_intersection

    def render(self):
        self.screen.fill(self.WHITE)

        for angle in range(360):
            ray_start = (self.predator_x, self.predator_y)
            ray_angle = angle
            closest_intersection = self.is_ray_blocked(ray_start, ray_angle)

            if closest_intersection is not None:
                ray_end_x = ray_start[0] + closest_intersection * math.cos(math.radians(ray_angle))
                ray_end_y = ray_start[1] + closest_intersection * math.sin(math.radians(ray_angle))
                pygame.draw.line(self.screen, self.RED, ray_start, (ray_end_x, ray_end_y), 1)
            else:
                ray_end_x = ray_start[0] + 1000 * math.cos(math.radians(ray_angle))
                ray_end_y = ray_start[1] + 1000 * math.sin(math.radians(ray_angle))
                pygame.draw.line(self.screen, self.BLUE, ray_start, (ray_end_x, ray_end_y), 1)

        pygame.draw.circle(self.screen, self.RED, (self.predator_x, self.predator_y), self.predator_radius)

        for object_name, object_vertices in self.objects.items():
            pygame.draw.polygon(self.screen, self.GREEN, object_vertices)

        pygame.display.flip()

    def run(self):
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

            keys = pygame.key.get_pressed()
            if keys[pygame.K_LEFT]:
                self.predator_x -= self.move_step
            if keys[pygame.K_RIGHT]:
                self.predator_x += self.move_step
            if keys[pygame.K_UP]:
                self.predator_y -= self.move_step
            if keys[pygame.K_DOWN]:
                self.predator_y += self.move_step

            self.render()

        pygame.quit()
        sys.exit()

if __name__ == "__main__":
    game = GameEnv(800, 600)
    game.run()


SystemExit: 

In [48]:
import pygame
import sys
import math

# Initialize Pygame
pygame.init()

# Set up display
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Predator Agent Environment")

# Define colors
RED = (255, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# Predator agent position and size
predator_x, predator_y = WIDTH // 2, HEIGHT // 2
predator_radius = 10
move_step = 1

# Define objects with arbitrary shapes
objects = {
    # "wall1": [(100, 100), (120, 100), (120, 200), (100, 200)],
    "wall2": [(200, 300), (220, 300), (220, 400), (200, 400)],
    "wall3": [(300, 200), (320, 200), (320, 300), (300, 300)],
    "circular_wall": [(400, 400), (430, 400), (430, 430), (400, 430)],
    "special_point": (100, 100)  # This is now a single point
}

# Function to check ray-object intersection
def does_agent_see_special_point(ray_start, ray_angle, special_point, objects):
    x1, y1 = ray_start
    x2, y2 = x1 + 1000 * math.cos(math.radians(ray_angle)), y1 + 1000 * math.sin(math.radians(ray_angle))

    for object_name, object in objects.items():
        if object_name == "special_point":
            continue  # Skip the special_point, as it's not a polygon

        for i in range(len(object)):
            x3, y3 = object[i]
            x4, y4 = object[(i + 1) % len(object)]

            denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

            if denominator == 0:
                continue

            t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denominator
            u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denominator

            if 0 <= t <= 1 and 0 <= u <= 1:
                intersection_x = x1 + t * (x2 - x1)
                intersection_y = y1 + t * (y2 - y1)

                # Check if the intersection point is the special point
                if (abs(intersection_x - special_point[0]) < 1 and
                        abs(intersection_y - special_point[1]) < 1):
                    return True  # Agent sees the special point

    return False  # Agent does not see the special point

# Main game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        predator_x -= move_step
    if keys[pygame.K_RIGHT]:
        predator_x += move_step
    if keys[pygame.K_UP]:
        predator_y -= move_step
    if keys[pygame.K_DOWN]:
        predator_y += move_step

    screen.fill(WHITE)

    for angle in range(360):
        ray_start = (predator_x, predator_y)
        ray_angle = angle

        sees_special_point = does_agent_see_special_point(ray_start, ray_angle, objects["special_point"], objects)

        ray_end = (ray_start[0] + 1000 * math.cos(math.radians(ray_angle)),
                   ray_start[1] + 1000 * math.sin(math.radians(ray_angle)))

        if sees_special_point:
            color = RED
        else:
            color = BLUE

        pygame.draw.line(screen, color, ray_start, ray_end, 1)

    pygame.draw.circle(screen, RED, (predator_x, predator_y), predator_radius)

    for object_name, object in objects.items():
        if object_name == "special_point":
            continue
        pygame.draw.polygon(screen, GREEN, object)

    pygame.display.flip()

pygame.quit()
sys.exit()


SystemExit: 

In [40]:
pygame.quit()
